In [1]:
# In[1]
# Import Libraries and Tools
import os
import sys
import pandas as pd
import numpy as np
import glob
import skimage.morphology
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import update_pointing, register
import matplotlib.pyplot as plt

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5, acweRestoreScale
from Metrics import JaccardIndexMetric as jim

In [ ]:
# Key Varibles

# Dataset
dataFolder  = '/home/jgra/Coronal Holes/newDataset/' # Update to reflect Dataset Location
traceFolder = os.path.join(ROOT_DIR, 'DatasetTools/DownloadLists/')
CR = 'CR2099'  # Update to reflect chosen Carrington Rotation

# ACWE Folders
ACWEmixFolder = '/mnt/coronal_holes/Paper 2/Code 02 Observations/HMI_Experiments/TestSeedingMethods/Standard/Uni/'
compareFolder = '/mnt/coronal_holes/Paper 2/Code 02 Observations/HMI_Experiments/TestEvolutionMethods/Standard/Uni/'

# ACWE Parameters 
acweChoice = '193'
hmiChoice  = 'Magnetogram'

# Inform user
verbose = True

# Tracibility
fileSufix = '.HMItypeDiscrepancies.csv'
resultsFolder = 'HMI_Experiments/TestSeedingMethods/Analysis/Results/'
resultsFolder = os.path.join(ROOT_DIR,resultsFolder)

In [ ]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

# Open & Align Magnetogram
def openHMI(filename,Ieuv,Heuv):
    
    # Open Magnetogram
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') #necessary for successful data read
    h_mag = hdulist[1].header
    J_mag = hdulist[1].data
    hdulist.close()

    # Create Maps
    mapHMI = sunpy.map.Map((J_mag,h_mag))
    mapHMI.plot_settings['cmap']='hmimag'
    mapACWE = sunpy.map.Map(Ieuv,Heuv)

    # Reproject Magnetogram
    hmiReproject = mapHMI.reproject_to(mapACWE.wcs)
    # hmiReproject,footprint = reproject_interp(mapHMI,mapACWE.wcs,
    #                                           mapACWE.data.shape)
    I = hmiReproject.data
    # Undo Keword Renaming
    H = dict()
    for k in hmiReproject.meta.keys(): 
        H[k.upper()] = hmiReproject.meta[k] 
    
    return I,H # hmiReproject

In [ ]:
# In[4]
# Open files and get list of images

# Open Carrington Rotation Document
CarringtonFile = traceFolder + CR + '.csv'
data = pd.read_csv(CarringtonFile,header=0)
keys = data.keys()

# Find Image groups we want 
acweChoice = np.where(keys == acweChoice)[0][0]
hmiData    = np.where(keys == hmiChoice)[0][0]

# Open Tracibility File
traceFile = resultsFolder + CR +fileSufix
tData = pd.read_csv(traceFile,header=0)
tKeys = tData.keys()

In [ ]:
# Prepare for analysis

# Create or find save location
crSaveFolder1 = ACWEmixFolder  + CR + '/'
crSaveFolder2 = compareFolder  + CR + '/'

In [ ]:
# Cycle through Dataset
for i in (tData[tKeys[0]]):
    
    # Get file
    file = data[keys[acweChoice]][i]
        
    # Find Images
    acweFolder = file.split('/')[0] + '/'
    mixImage = glob.glob(crSaveFolder1+acweFolder+'*'+os.path.basename(file)+'*')[0]
    cmpImage = glob.glob(crSaveFolder2+acweFolder+'*'+os.path.basename(file)+'*')[0]
    
    # Open Files
    A,AM,SegM = acweSaveSeg_v5.openSeg(mixImage)
    _,AC,SegC = acweSaveSeg_v5.openSeg(cmpImage)
    
    # Resize Segmentations
    SegM = acweRestoreScale.upscale(SegM,AM)
    SegC = acweRestoreScale.upscale(SegC,AC)
    
    # Label CHs
    segm = skimage.morphology.dilation(SegM.astype(bool),
                                       np.ones([40,40]))
    segm = skimage.measure.label(segm,connectivity=2)
    segm = (segm * SegM).astype(int)
    
    segc = skimage.morphology.dilation(SegC.astype(bool),
                                       np.ones([40,40]))
    segc = skimage.measure.label(segc,connectivity=2)
    segc = (segc * SegC).astype(int)
    
    # Get Magnetogram Data
    Itmp,Idsp,Htmp = openAIA(dataFolder+str(CR) +'/'+file)
    I,H = openHMI(dataFolder + CR + '/' + data[keys[hmiData]][i],Itmp,Htmp)
    
    # Plot
    plt.figure(figsize = [15,5],dpi=300)
    plt.rcParams.update({'font.size': 11})
    plt.suptitle(os.path.basename(file))
    plt.subplot(1,3,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap='gray')
    plt.title('EUV Observation')
    plt.axis(False)
    plt.subplot(1,3,2)
    plt.imshow(np.flip(SegC,axis=0),interpolation='None')
    plt.title('Old method')
    plt.axis(False)
    plt.subplot(1,3,3)
    plt.imshow(np.flip(SegM,axis=0),interpolation='None')
    plt.title('New method')
    plt.axis(False)
    plt.show()
    
    # Calculate Unipolarity of Region
    for j in range(1,np.max(segc) + 1):
        
        IOUlast = 0
        kept    = 0
        for k in range(1,np.max(segm) + 1):
            
            IOU = jim.IOU(segc==j,segm==k,True)
            
            if IOU>IOUlast:
                kept = k * 1
                IOUlast = IOU * 1
        

        # Flatten Selected Region old method
        flattenedc = I[segc==j].flatten()
        # Remove NAN Values, if any Persist
        flattenedc = flattenedc[np.logical_not(np.isnan(flattenedc))]
        # Calculate Unweighted Unipolarity
        numc = np.mean(np.abs(flattenedc)) - np.abs(np.mean(flattenedc))
        denc = np.mean(np.abs(flattenedc))
        unic = numc/denc
        
        if kept != 0:
            # Flatten Selected Region old method
            flattenedm = I[segm==kept].flatten()
            # Remove NAN Values, if any Persist
            flattenedm = flattenedm[np.logical_not(np.isnan(flattenedm))]
            # Calculate Unweighted Unipolarity
            numm = np.mean(np.abs(flattenedm)) - np.abs(np.mean(flattenedm))
            denm = np.mean(np.abs(flattenedm))
            unim = numm/denm
            
            print('Region:', j, 'old Uni:', unic, 'new Uni:', unim)
        else:
            print('Region:', j, 'old Uni:', unic, 'new: Missing')